In [7]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import silhouette_score
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt

In [8]:
df = pd.read_csv('./data/good_data/fraudTrain.csv')

In [9]:
# Drop Unecessary Data
# First, last name, job, dob unecessary PII
# Get rid of labels for unsupervised learning (is_fraud)
# CC_Num, trans_num unique identifier so not necessary
# Merchant: Already have location so not necessary
# Street, city, and zip not necessary since we have lat and lng

unecessary_columns = [
    'first', 
    'last', 
    'cc_num', 
    'merchant',
    'is_fraud',
    'street', 
    'city', 
    'zip', 
    'city_pop',
    'job',
    'dob',
    'trans_num',
    'trans_date_trans_time',
    'unix_time',
    'state',
    'Unnamed: 0'
]
unsupervised_df = df.drop(columns=unecessary_columns)

le = LabelEncoder()
unsupervised_df['category'] = le.fit_transform(unsupervised_df['category'])
unsupervised_df['gender'] = le.fit_transform(unsupervised_df['gender'])

In [10]:
scalar = StandardScaler()
pc_df = scalar.fit_transform(unsupervised_df)

In [11]:
x_train, x_test = train_test_split(pc_df, test_size=0.2)

In [ ]:
# Using GMM
EM = GaussianMixture(n_components=2)
EM.fit(x_train)

print(silhouette_score(x_test, EM.predict(x_test)))